# Transfer Learning Code for the Traveling Salesman Computer Vision Kaggle Dataset

This workbook is a starter code for the Traveling Salesman computer vision dataset. This is a more advanced notebook that uses transfer learning; however, it is not particularly optimized.  It is also possable to run this project from Google CoLab.  

In [1]:
import numpy as np
import pandas as pd
import os

PATH = "tsp-cv"
PATH_TRAIN = os.path.join(PATH, "train.csv")

Next we check versions and if the GPU is available. A GPU or TPU will be very helpful.

In [2]:
# What version of Python do you have?
import sys

import keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if tf.config.list_physical_devices('GPU') \
      else "NOT AVAILABLE")



Tensor Flow Version: 2.13.1
Keras Version: 2.13.1

Python 3.10.0 (tags/v3.10.0:b494f59, Oct  4 2021, 19:00:18) [MSC v.1929 64 bit (AMD64)]
Pandas 2.2.1
Scikit-Learn 1.4.2
GPU is NOT AVAILABLE


Next, we prepare to read the training data (that we have labels for) and the test data that we must predict and send to Kaggle.

In [3]:
df_train = pd.read_csv(PATH_TRAIN)

We want to use early stopping. To do this, we need a validation set. We will break the data into 80 percent test data and 20 validation.

In [4]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")

Training size: 14416
Validate size: 1602


Next, we create the generators that will provide the images to the neural network as it is trained.  We normalize the images so that the RGB colors between 0-255 become ratios between 0 and 1.  We also use the **flow_from_dataframe** generator to connect the Pandas dataframe to the actual image files. We see here a straightforward implementation; you might also wish to use some of the image transformations provided by the data generator.

The **HEIGHT** and **WIDTH** constants specify the dimensions that the image will be scaled (or expanded) to. It is probably not a good idea to expand the images.

In [5]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 256
HEIGHT = 256

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=32, # Keeping the training batch size small USUALLY increases performance
        class_mode='raw')

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
        dataframe=df_validate_cut,
        directory=PATH,
        x_col="filename",
        y_col="distance",
        target_size=(HEIGHT, WIDTH),
        batch_size=256, # Make the validation batch size as large as you have memory for
        class_mode='raw')

Found 14416 validated image filenames.
Found 1602 validated image filenames.


We now create the neural network and fit it.  Some essential concepts are going on here.

* **Batch Size** - The number of training samples that should be evaluated per training step.  Smaller batch sizes, or mini-batches, are generally preferred.
* **Step** - A training step is one complete run over the batch.  At the end of a step, the weights are updated, and the neural network learns.
* **Epoch** - An arbitrary point at which to measure results or checkpoint the model.  Generally, an epoch is one complete pass over the training set.  However, when generators are used, the training set size is theoretically infinite. Because of this, we set a **steps_per_epoch** parameter.
* **validation steps** - The validation set may also be infinite; because of this, we must specify how many steps we wish to validate at the end of each Epoch.

## Transfer Learning

We will now use a ResNet neural network as a basis for our neural network.  We will redefine both the input shape and output of the ResNet model, so we will not transfer the weights.  Since we redefine the input; the weights are of minimal value.  We begin by loading, from Keras, the ResNet50 network.

In [6]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Input

input_tensor = Input(shape=(HEIGHT, WIDTH, 3))

base_model = ResNet50(
    include_top=False, weights=None, input_tensor=input_tensor,
    input_shape=None)

#base_model.summary()

Now we must add a few layers to the end of the neural network so that it becomes a regression model.

In [7]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
model=Model(inputs=base_model.input,outputs=Dense(1)(x))

Now we train just like before, the only difference is that we do not define the entire neural network here.

In [8]:
from keras.callbacks import EarlyStopping
from keras.metrics import RootMeanSquaredError

# Important, calculate a valid step size for the validation dataset
STEP_SIZE_VALID=val_generator.n//val_generator.batch_size

model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError(name="rmse")])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto',
        restore_best_weights=True)
with tf.device('GPU:0'):
    history = model.fit(train_generator, epochs=100, steps_per_epoch=50, 
                        validation_data = val_generator, callbacks=[monitor],
                        verbose = 1, validation_steps=STEP_SIZE_VALID)

Epoch 1/100
50/50 [==============================] - 487s 9s/step - loss: 365266848.0000 - rmse: 19111.9551 - val_loss: 4389396992.0000 - val_rmse: 66252.5234
Epoch 2/100
10/50 [=====>........................] - ETA: 5:22 - loss: 124853744.0000 - rmse: 11173.7969

KeyboardInterrupt: 

In [10]:
model.save("model.keras")

In [11]:
model = keras.models.load_model("model.keras")

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 262, 262, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 128, 128, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 128, 128, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          